# Time series analyses on stock prices and building a model for prediction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:

dataset_train = pd.read_csv('NSE-TATAGLOBAL.csv')
training_set = dataset_train.iloc[:, 1:2].values

In [3]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)


In [5]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout



In [7]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
62/62 [==============================] - 6s 104ms/step - loss: 0.0079
Epoch 2/100
62/62 [==============================] - 6s 99ms/step - loss: 0.0030
Epoch 3/100
62/62 [==============================] - 6s 102ms/step - loss: 0.0031
Epoch 4/100
62/62 [==============================] - 6s 99ms/step - loss: 0.0024
Epoch 5/100
62/62 [==============================] - 6s 104ms/step - loss: 0.0024
Epoch 6/100
62/62 [==============================] - 6s 100ms/step - loss: 0.0023
Epoch 7/100
62/62 [==============================] - 6s 102ms/step - loss: 0.0023
Epoch 8/100
62/62 [==============================] - 6s 99ms/step - loss: 0.0024
Epoch 9/100
62/62 [==============================] - 7s 108ms/step - loss: 0.0022
Epoch 10/100
62/62 [==============================] - 7s 118ms/step - loss: 0.0019
Epoch 11/100
62/62 [==============================] - 7s 115ms/step - loss: 0.0018
Epoch 12/100
62/62 [==============================] - 8s 122ms/step - loss: 0.0017
Epoch 13/100
62/

62/62 [==============================] - 7s 120ms/step - loss: 5.8285e-04
Epoch 98/100
62/62 [==============================] - 7s 109ms/step - loss: 5.7028e-04
Epoch 99/100
62/62 [==============================] - 6s 104ms/step - loss: 7.4354e-04
Epoch 100/100
62/62 [==============================] - 7s 108ms/step - loss: 6.3864e-04


In [8]:
dataset_test = pd.read_csv('tatatest.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values



FileNotFoundError: [Errno 2] File tatatest.csv does not exist: 'tatatest.csv'

In [ ]:
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 76):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
plt.plot(real_stock_price, color = 'black', label = 'TATA Stock Price')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted TATA Stock Price')
plt.title('TATA Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('TATA Stock Price')
plt.legend()
plt.show()